In [ ]:
# Transformer 모델내에서 사용되는 워드 임베딩 처리 및 학습
# 학습 목표 
# - 각 단어마다 vocab 전체와 확률 비교 → 정답과 비교 → 손실 계산 → 파라미터 업데이트 → logits 생성이라는 흐름으로 학습
# - 모델학습 과정에서 임베딩이 점점 의미를 반영하게 되고, 비슷한 단어끼리 가까워지는 성질이 생긴다
# - 임베딩 weight 업데이트, 임베딩 행렬의 각 벡터가 학습을 통해 의미 공간에서 위치를 바꾸는 것이다. 
# - 임베딩은 토큰화되어진 문자를 벡터로 변환하는것이 아니다

# 1. 토크나이저 -> 인덱스 변환 
# - 텍스트를 토큰 단위로 분리 (WordPiece, BPE, SentencePiece 등) 
# - 각 토큰을 고유 인덱스로 매핑
# 2. 임베딩 레이어 생성 
# - PyTorch의 nn.Embedding을 사용해 인덱스를 고정 길이 벡터로 변환 
# - 학습 가능한 파라미터로 초기화 -> 학습 과정에서 업데이트 됨 
# 3. 학습 루프 및 임베딩 학습 방식 
# - 랜덤 초기화 후 학습 : 모델 학습 과정에서 임베딩이 점차 의미를 학습 
# - 사전학습 임베딩 활용 : Word2Vec, GloVe, FastText 같은 사전학습 벡터를 초기값으로 사용 
# - Transformer 기반 임베딩 : BERT, GPT 등 사전학습 모델의 임베딩 레이어를 가져와 파인 튜닝

In [1]:
# 1. 토크나이저 -> 인덱스 변환 
# - 텍스트를 토큰 단위로 분리 (WordPiece, BPE, SentencePiece 등) 
# - 각 토큰을 고유 인덱스로 매핑
from transformers import AutoTokenizer

# 작은 코퍼스
corpus = [
    "안녕하세요 오늘은 날씨가 맑습니다",
    "저는 자연어 처리를 공부하고 있습니다",
    "임베딩은 단어를 벡터로 표현하는 방법입니다",
    "파이토치는 딥러닝을 위한 강력한 라이브러리입니다",
    "언어 모델은 다음 단어를 예측하는 방식으로 학습합니다",
    "작은 데이터셋으로도 실험을 시작할 수 있습니다",
    "머신러닝은 데이터를 통해 패턴을 학습합니다",
    "딥러닝은 인공신경망을 기반으로 합니다",
    "토큰화는 문장을 단어 단위로 나누는 과정입니다",
    "모델은 입력을 받아 출력을 생성합니다",
    "하이퍼파라미터는 학습 성능에 큰 영향을 줍니다",
    "에포크는 전체 데이터셋을 한 번 학습하는 단위를 의미합니다",
    "배치 크기는 한 번에 처리하는 샘플 수입니다",
    "손실 함수는 모델의 예측과 정답의 차이를 측정합니다",
    "옵티마이저는 파라미터를 업데이트하는 알고리즘입니다",
    "학습률은 파라미터를 얼마나 크게 조정할지 결정합니다",
    "정규화는 과적합을 방지하는 방법입니다",
    "드롭아웃은 일부 뉴런을 무작위로 끊어 학습을 안정화합니다",
    "GPU는 대규모 연산을 빠르게 수행할 수 있습니다",
    "실험을 반복하면 더 나은 결과를 얻을 수 있습니다"
]

# 사전학습된 gpt2 모델의 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# gpt2 토크나이저 모델에서는 padding 토큰(pad_token)을 정의하지 않는다. 문자을 끝날때 EOS(end of sequence) 토큰만 사용
# 배치 학습시 길이 맞추기, 여러 문장을 한 배치로 학습하려면 길이를 맞추어야 한다
# 짧은 문장은 padding을 넣어 길이를 맞추는데, gpt2 모델에는 pad 토큰이 없으니 EOS 토큰을 대신 사용한다
# gpt2 모델은 pad 토큰을 따로 처리하지 않기 때문에, pad를 eos로 설정하면 모델이 '문장이 끝났다'는 의미로 자연스럽게 인식한다
# loss 계산시 pad 토큰을 무시하거나 특별히 다루지 않아도 된다
tokenizer.pad_token = tokenizer.eos_token

# 토큰화, padding=True 가장 긴 문장 길이에 맞추어 나머지 문장들을 pad 토큰으로 채워준다
inputs = tokenizer(corpus, return_tensors='pt', padding=True)

input_ids = inputs['input_ids'] 
vocab_size = tokenizer.vocab_size # 사전학습된 gpt2 모델의 토크나이저 vocabulary 50257 보유
embed_dim = 128 # embedding 128 차원

# 결과 확인
# print(inputs)
print(inputs['input_ids'].shape)
print(vocab_size)
# print(inputs['attention_mask'])

torch.Size([20, 69])
50257


In [3]:
# 임베딩 공간내 벡터 위치 학습 모델
import torch
import torch.nn as nn

class SimpleLM(nn.Module):
    # vocabulary 50257, embed_dim 128
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim) # 각 단어 ID를 벡터로 변환
        self.decoder = nn.Linear(embed_dim, vocab_size) # 임베딩 벡터를 vocab 크기 만큼 logits으로 변환
    
    # 모델의 학습과정을 반복하면 학습을 통해 의미 기반 벡터 공간을 형성하는게 목적이다
    # 비슷한 의미/맥락의 단어 -> 임베딩 공간에서 서로 가까워지며, 다른 의미/맥락의 단어 -> 서로 떨어짐
    def forward(self, x):
        emb = self.embedding(x)
        logits = self.decoder(emb)
        return logits

In [4]:
# 모델, 손실함수, 옵티마이저 객체 생성
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'PyTorch Version: {torch.__version__}, Device: {device}')

model = SimpleLM(vocab_size=vocab_size, embed_dim=embed_dim).to(device) # vocabulary 50257, embed_dim 128
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

PyTorch Version: 2.8.0+cu129, Device: cuda


In [ ]:
# 학습 진행

# next-token prediction: 입력과 라벨 어긋나게
inputs_shift = input_ids[:, :-1].to(device)
labels_shift = input_ids[:, 1:].to(device)

for epoch in range(2000):
    optimizer.zero_grad()    
    logits = model(inputs_shift)
    # print(f'logits: {logits}, logits.shape: {logits.shape}')
    # print(f'labels_shift: {labels_shift}, labels_shift.shape: {labels_shift.shape}')

    # loss = loss_fn(logits.view(-1, vocab_size), labels_shift.view(-1))
    # 학습 루프에서는 .reshape()를 쓰는 게 더 안정적
    # 1. logits -> Softmax -> 확률 분포 변환, 2. CrossEntropyLoss 정답 토큰 ID와 예측 확률 분포를 비교
    loss = loss_fn(logits.reshape(-1, vocab_size), labels_shift.reshape(-1))
    # print(logits.shape, labels_shift.shape)
    # print(logits.reshape(-1, vocab_size).shape, labels_shift.reshape(-1).shape)

    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib import rc
from adjustText import adjust_text

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지

# 관심 단어
words = ['이재명', '대통령', '인공지능', 'AI', 'GPU']

# 토큰 ID 변환
word_ids = [ tokenizer.encode(w, add_special_tokens=False)[0] for w in words ]

# 학습된 임베딩 벡터 추출
vectors = model.embedding.weight[word_ids].detach().cpu().numpy()

# PCA로 2차원 축소
pca = PCA(n_components=2) # 차원을 2개로 줄이겠다는 의미, 고차원 데이터를 시각화 또는 중요한 축만 남겨서 분석하는 목정
reduced = pca.fit_transform(vectors) # 변환, 예시) vector (5, 7678) -> (5, 2)

# 시각화
plt.figure(figsize=(8,6))
plt.scatter(x=reduced[:, 0], y=reduced[:, 1], s=50) # x좌표, y좌표, 점의 크기는 50

texts = []
for i, word in enumerate(words):
    texts.append(plt.text(x=reduced[i,0], y=reduced[i,1], s=word, fontsize=12))

# 라벨 자동 조정
adjust_text(texts=texts, arrowprops=dict(arrowstyle="->", color='gray'))

plt.title("학습된 단어 임베딩 PCA 시각화 (라벨 겹침 방지)")
plt.show()

In [ ]:
# 학습된 단어 임베딩 PCA 시각화
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib import rc
from adjustText import adjust_text

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지


# 관심 단어들
words = ["학교","학생","교실","커피","카페","에스프레소","강아지","고양이","동물"]

# 토큰 ID 변환
word_ids = [tokenizer.encode(w, add_special_tokens=False)[0] for w in words]

# 학습된 임베딩 벡터 추출
vectors = model.embedding.weight[word_ids].detach().cpu().numpy()

# PCA로 2차원 축소
pca = PCA(n_components=2)
reduced = pca.fit_transform(vectors)

# 시각화
plt.figure(figsize=(8,6))
plt.scatter(reduced[:,0], reduced[:,1], s=50)

texts = []
for i, word in enumerate(words):
    texts.append(plt.text(reduced[i,0], reduced[i,1], word, fontsize=12))

# 라벨 자동 조정
adjust_text(texts, arrowprops=dict(arrowstyle="->", color='gray'))

plt.title("학습된 단어 임베딩 PCA 시각화 (라벨 겹침 방지)")
plt.show()

In [ ]:
# 학습된 단어 임베딩 t-SNE 시각화
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from adjustText import adjust_text

# 관심 단어들
words = ["학교","학생","교실","커피","카페","에스프레소","강아지","고양이","동물"]

# 토큰 ID 변환
word_ids = [tokenizer.encode(w, add_special_tokens=False)[0] for w in words]

# 학습된 임베딩 벡터 추출
vectors = model.embedding.weight[word_ids].detach().cpu().numpy()

# t-SNE로 2차원 축소
tsne = TSNE(n_components=2, random_state=42, perplexity=5, max_iter=1000)
reduced = tsne.fit_transform(vectors)

# 시각화
plt.figure(figsize=(8,6))
plt.scatter(reduced[:,0], reduced[:,1], s=50)

texts = []
for i, word in enumerate(words):
    texts.append(plt.text(reduced[i,0], reduced[i,1], word, fontsize=12))

# 라벨 자동 조정
adjust_text(texts, arrowprops=dict(arrowstyle="->", color='gray'))

plt.title("학습된 단어 임베딩 t-SNE 시각화 (라벨 겹침 방지)")
plt.show()

In [ ]:
# 코사인 유사도 행렬
from sklearn.metrics.pairwise import cosine_similarity

# 관심 단어들
words = ["파이토치","딥러닝","GPU"]

# 토큰 ID 변환
word_ids = [tokenizer.encode(w, add_special_tokens=False)[0] for w in words]

# 학습된 임베딩 벡터 추출
vectors = model.embedding.weight[word_ids].detach().cpu().numpy()

# 코사인 유사도 계산
sim_matrix = cosine_similarity(vectors)
print("코사인 유사도 행렬:")
print(sim_matrix)

In [ ]:
# 파이토치-딥러닝-GPU 임베딩 위치 (PCA)
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from adjustText import adjust_text

# PCA로 2차원 축소
pca = PCA(n_components=2)
reduced = pca.fit_transform(vectors)

# 시각화
plt.figure(figsize=(6,5))
plt.scatter(reduced[:,0], reduced[:,1], s=80)

texts = []
for i, word in enumerate(words):
    texts.append(plt.text(reduced[i,0], reduced[i,1], word, fontsize=12))

adjust_text(texts, arrowprops=dict(arrowstyle="->", color='gray'))
plt.title("파이토치-딥러닝-GPU 임베딩 위치 (PCA)")
plt.show()